In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage
from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.activity as activity

%load_ext autoreload
%autoreload 2

Appended root directory /media/aleksejs/DataHDD/work/codes/comp-neuro/analysis-mesoscopic/pub-2020-exploratory-analysis


In [2]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
# params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [3]:
dataDB = DataFCDatabase(params)

Reading channel label file
Reading channel area file
Reading allen brain map
Reading task structure
Reading session structure
Searching for data files
Found mice ['mou_5', 'mou_6', 'mou_7', 'mou_9']


In [4]:
ds = DataStorage('gallerosalas_result_activity.h5')

In [5]:
mc = MetricCalculator(serial=True, verbose=False)

In [6]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

mice {'mou_9', 'mou_6', 'mou_5', 'mou_7'}
nSessions 4
datatypes ['raw', 'bn_session', 'bn_trial']
nChannel 27


In [7]:
dataDB.calc_shortest_distances()

## Compute mean over time for all mice and condition combinations

In [16]:
exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
]

In [19]:
activity.compute_mean_interval(dataDB, ds, {'Hit', 'CR'}, exclQueryLst=exclQueryLst)

['mou_5', 'PRE', 'raw', 'Hit']
Skipping existing mean_2017_03_06_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_06_session02_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_13_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_13_session02_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_14_session02_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_15_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_16_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_16_session02_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_16_session03_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_17_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_22_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_22_session03_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_22_session04_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_23_session01_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_23_session02_mou_5_PRE_raw_Hit
Skipping existing mean_2017_03_24_sessi

# 1. Significance

## 1.1 Top 10 most significant connections for Hit/Cr, Naive/Expert

## 1.3. Accuracy of hit vs cr prediction by mouse, phase

# 2. Consistency

## 2.1. Consistency of channel activity

* DF/F should be independent of channel brightness/opacity, since it normalizes baseline.
* Thus, should be able to compare DFF of different mice directly

## 2.2 Channels significant for Hit/Cr. Overlap by mouse

1. Loop over datatype, interval, session
2. Test each channel for Hit/Cr
3. Count significant sessions for each channel
4. Scatter counts

**TODO**
* Split by Naive/Expert

In [23]:
activity.plot_consistency_significant_activity_byaction(dataDB, ds)

['bn_session', 'bn_trial', 'raw'] ['DEL', 'PRE', 'REW', 'TEX']


## 2.3. Channels significant for phase. Overlap by mouse

In [31]:
activity.plot_consistency_significant_activity_byphase(dataDB, ds, ['TEX', 'DEL'])

['bn_session', 'bn_trial', 'raw'] ['CR', 'Hit']
